In [93]:
using Kiro24Base
filename = "tiny.json"
instance = Kiro24Base.read_instance(filename, verbose=true)

Instance(name = tiny,
two-tone delta = 2,
vehicles (5) = [Vehicle(id=1, type=regular),Vehicle(id=2, type=regular),Vehicle(id=3, type=two-tone),Vehicle(id=4, type=regular),Vehicle(id=5, type=two-tone)],
shops = [Shop(name = body,
	isPaint = false,
	resequencingLag = 1,
	resequencingCost = 20.0,
	batchSizes (1) = [BatchSize(id=1, cost=2.0, min_size=2, max_size=4, vehicles=[1, 2, 4])],
	lotChanges (0) = [],
	rollingWindows (0) = []
),
Shop(name = paint,
	isPaint = true,
	resequencingLag = 2,
	resequencingCost = 20.0,
	batchSizes (0) = [],
	lotChanges (1) = [LotChange(id=2, cost=3.0, partition=[[1, 2], [3, 4, 5]])],
	rollingWindows (0) = []
),
Shop(name = assembly,
	isPaint = false,
	resequencingLag = 0,
	resequencingCost = 20.0,
	batchSizes (2) = [BatchSize(id=4, cost=3.0, min_size=2, max_size=3, vehicles=[1, 2, 3]),BatchSize(id=5, cost=3.0, min_size=1, max_size=2, vehicles=[4, 5])],
	lotChanges (0) = [],
	rollingWindows (1) = [RollingWindow(id=3, cost=2.0, window_size=3, max_vehicles=2, 

In [94]:
function transform_filename(input::String)::String
    # Extract the base name without the .json extension
    base_name = replace(input, ".json" => "")
    
    # Append "_sol" to the base name
    transformed_name = base_name * "_sol.json"
    
    # Prepend "data/" to the transformed name
    return "data/" * transformed_name
end


transform_filename (generic function with 1 method)

In [95]:
file_path = transform_filename(filename)


"data/tiny_sol.json"

In [96]:
using JSON,Random
using OrderedCollections

# Specify the file name and location
# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = false  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]
two_tone_vehicle = []
i = 0
for vehicle in vehicles
    i+=1
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: ")
    else
        # Otherwise, print the actual vehicle type
        if vehicle["type"] == "two-tone"
            push!(two_tone_vehicle,i)
        end
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end



In [97]:
cs = parameters["resequencing_cost"]
number_of_shops = length(shops)    
δ = parameters["two_tone_delta"]
println(δ)
i=0 
for shop in shops
    i+=1
    
    println("Shop: ",i,"  ", shop)
end
println("Number of shops: ", number_of_shops)
number_of_vehicles = length(vehicles)
println("number_of_vehicles, ",number_of_vehicles)

In [98]:

# Initialize an empty dictionary to store the mapping
shop_map = OrderedDict{String, Vector{Any}}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
lag_map = OrderedDict{String, Integer}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
# Populate the shop_map with shop names as keys and empty arrays as values
i = 0
for shop in shops
    i+=1
    shop_map[shop["name"]] = []
    lag_map[shop["name"]] = shop["resequencing_lag"]
    println(shop["name"],"i :",i)
end
n = number_of_vehicles
println("n: ", n)
# Initialize counters or variables if needed

# Assume `model` is defined somewhere before this loop (like in JuMP)
# Example: model = Model(Ipopt.Optimizer)

# Iterate through constraints and associate them with the corresponding shop in shop_map

for constraint in constraints
    name = constraint["shop"]
    if haskey(shop_map, name)
        push!(shop_map[name], constraint)
    else
        println("Warning: Shop name $name in constraints does not exist in shops.")
    end
end

(lag_map)


OrderedDict{String, Integer} with 3 entries:
  "body"     => 1
  "paint"    => 2
  "assembly" => 0

In [99]:
n

5

In [100]:
# Function to generate targeted neighbors
function generate_targeted_neighbors0(perm, Vr, Ul,Vb,Mb,mb)
    # Start with a random perturbation as a fallback

    perturbed_perm = random_perturbation(perm)
    n = length(perturbed_perm)
    k = 0

    # Handle perturbation based on Ul
    l = length(Ul)
    if l>0 
        for j in 1:l
            ul = Ul[j]
            for i in 1:(n-1)
                # Check if the consecutive elements are not in the same sublist in Ul
                if !in_same_lot(perturbed_perm[i], perturbed_perm[i + 1], ul)
                    # Replace one of the elements with a random element from perm
                    for j in i+1:n
                        if j != i && j != i + 1 && in_same_lot(perturbed_perm[i], perturbed_perm[j], ul)
                            # Replace element at index i with the one at index j
                            perturbed_perm[i],perturbed_perm[j] = perturbed_perm[j],perturbed_perm[i]
                            break  # Exit loop once a change has been made
                        end
                    end
                end
            end
        end
    end
    return perturbed_perm
end
function generate_targeted_neighbors1(perm, Vr, Ul,Vb,Mb,mb)
    perturbed_perm = random_perturbation(perm)
    n = length(perturbed_perm)
    r = length(Vr)
    k = 0
    n = length(perturbed_perm)
    if r > 0
        for j in 1:r
            vr = Set(Vr[j])
            for i in 1:n 
                if perturbed_perm[i] in vr
                    k += 1
                else
                    k = 0
                end

                if k == r && i < n
                    # Find the next element not in Vr
                    for j in i+1:n
                        if perturbed_perm[j] ∉ vr
                            # Swap the i-th element with the j-th element
                            
                            perturbed_perm[i], perturbed_perm[j] = perturbed_perm[j], perturbed_perm[i]
                            break
                        end
                    end
                    k = 0  # Reset k after the swap
                end
            end
        end
    end
    return perturbed_perm
end
function generate_targeted_neighbors2(perm, Vr, Ul,Vb,Mb,mb)
    perturbed_perm = random_perturbation(perm)
    n = length(perturbed_perm)
    r = length(Vr)
    k = 0
    n = length(perturbed_perm)
    # Batch size constraints (Vb, mb, Mb, cb)
    if !isempty(Vb) 
        for i in 1:length(Vb)
            vb = Set(Vb[i])
            Mb0 = Mb[i]
            mb0 = mb[i]
            
            for t in 1:n-1
                b = true
                if t > 1
                    b = !(perturbed_perm[t-1] in vb) && b
                end
                b  = perturbed_perm[t] in vb && b
                if !b 
                    continue
                end
                for t0 in t:n
                    for k in t:t0
                        b = (perturbed_perm[k] in vb) && b
                    end
                    if !b 
                        continue
                    end
                    if t0 <= n-1
                        b = !(perturbed_perm[t0+1] in vb) && b
                    end
                    if b
                        s = (t0-t) ÷ 2

                        if (t > 1 && ((t0 - t + 1) < mb0 || (t0 - t + 1) > Mb0))
                            perturbed_perm[t-1], perturbed_perm[t+s] = perturbed_perm[t+s], perturbed_perm[t-1];
                        elseif (t0 < n && ((t0 - t + 1) < mb0 || (t0 - t + 1) > Mb0)) 
                            perturbed_perm[t0+1], perturbed_perm[t+s] = perturbed_perm[t+s], perturbed_perm[t0+1];
                        end
                        
                    
                    end
                end
            end
        end
    end

    return perturbed_perm
end

# Helper function to check if two elements are in the same sublist in Ul
function in_same_lot(elem1, elem2, Ul)
    for lot in Ul
        if elem1 in lot && elem2 in lot
            return true
        end
    end
    return false
end


# Helper function to apply a random perturbation
function random_perturbation(perm)
    # Randomly choose two indices to swap
    copy_perm = copy(perm)
    idx1, idx2 = rand(1:length(perm)), rand(1:length(perm))
    copy_perm[idx1], copy_perm[idx2] = copy_perm[idx2], copy_perm[idx1]
    return copy_perm
end


random_perturbation (generic function with 1 method)

In [101]:
# Initialize result and Δs lists
result = []
Δs = []

# Initialize global constraint lists for all shops
Cl = []          # Cost list (Ints)
Ul = []  # Partitions (Array of arrays of Ints)
Vr = []  # Vehicles (Array of arrays of Ints)
Mr = []          # Max vehicles (Ints)
wr = []          # Window size (Ints)
cr = []          # Cost (Ints)
mb = []          # Min vehicles (Ints)
Mb = []          # Max vehicles (Ints)
cb = []          # Cost (Ints)
Vb = []  # Vehicles (Array of Ints)

# Loop through each shop and its constraints
for (shop, constraints) in shop_map
    println("shop: ", shop)
    
    # Initialize shop-specific constraint lists
    Cl0 = Int[]          # Cost list (Ints)
    Ul0 = Vector{Vector{Int}}[]  # Partitions (Array of arrays of Ints)
    Vr0 = Vector{Int}[]  # Vehicles (Array of arrays of Ints)
    Mr0 = Int[]          # Max vehicles (Ints)
    wr0 = Int[]          # Window size (Ints)
    cr0 = Int[]          # Cost (Ints)
    mb0 = Int[]          # Min vehicles (Ints)
    Mb0 = Int[]          # Max vehicles (Ints)
    cb0 = Int[]          # Cost (Ints)
    Vb0 = []  # Vehicles (Array of Ints)
    
    # Add the lag for the current shop
    push!(Δs, lag_map[shop])
    
    # Loop through constraints and separate them by type
    for constraint in constraints
        if constraint["type"] == "lot_change"
            cl = constraint["cost"]
            Ul_0 = constraint["partition"]
            push!(Ul0, Ul_0)  # Ensure Ul_0 is an array of Int
            push!(Cl0, cl)
        end

        if constraint["type"] == "rolling_window"
            Vr_0 = constraint["vehicles"]
            Mr_0 = constraint["max_vehicles"]
            wr_0 = constraint["window_size"]
            cr_0 = constraint["cost"]
            push!(Vr0, Vr_0)  # Ensure Vr_0 is an array of Int
            push!(Mr0, Mr_0)
            push!(wr0, wr_0)
            push!(cr0, cr_0)
        end

        if constraint["type"] == "batch_size"
            mb_0 = constraint["min_vehicles"]
            Mb_0 = constraint["max_vehicles"]
            Vb_0 = constraint["vehicles"]
            cb_0 = constraint["cost"]
            push!(mb0, mb_0)
            push!(Mb0, Mb_0)
            push!(Vb0, Vb_0)  # Ensure Vb_0 is an array of Int
            push!(cb0, cb_0)
        end
    end

    # Push all constraint lists (even if they are empty) to the global lists
    push!(Cl, Cl0)
    push!(Ul, Ul0)
    push!(Vr, Vr0)
    push!(Mr, Mr0)
    push!(wr, wr0)
    push!(cr, cr0)
    push!(mb, mb0)
    push!(Mb, Mb0)
    push!(cb, cb0)
    push!(Vb, Vb0)
end


In [102]:
Ul[2][1]


2-element Vector{Vector{Int64}}:
 [1, 2]
 [3, 4, 5]

In [103]:
file_path = transform_filename(filename)
solution = read_solution(instance::Instance,file_path)
current_cost = compute_cost(instance, solution, verbose=true)
# Compute the cost of the solution
total_cost = compute_cost(instance, solution, verbose=true)
println("Total Cost: ", total_cost)
is_feasible(instance, solution,verbose = true)
total_cost

3.0

In [105]:
function simulated_annealing(solution, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs; T0, alpha, max_iter)
    # Initialize variables
    current_cost = compute_cost(instance, solution, verbose=true)
    best_perms = transpose(copy(solution.entries))  # Best solution found
    best_cost = current_cost  # Best cost found
    T = T0  # Initial temperature
    counter = 0
    perms = transpose(copy(solution.entries))

    # Check matrix size to ensure valid indexing
    println("Matrix size: ", size(perms))  # Debugging step

    for iter in 1:max_iter
        if (T < 1e-100)
            break
        end
        neighbor_costs = []
        neighbor_perms = []
        # Generate a set of neighboring solutions (perturbations)
        for i in 1:size(perms, 1)
            perm_copy = copy(perms)
            perm = perms[i, :]
            
           """ perturbed_perm0 = generate_targeted_neighbors0(perm, Vr[i], Ul[i], Vb[i], Mb[i], mb[i])
            perturbed_perm1 = generate_targeted_neighbors1(perm, Vr[i], Ul[i], Vb[i], Mb[i], mb[i])
            perturbed_perm2 = generate_targeted_neighbors2(perm, Vr[i], Ul[i], Vb[i], Mb[i], mb[i])"""
           perturbed_perm0 = random_perturbation(perm)
            perturbed_perm1 = random_perturbation(perm)
            
            perturbed_perm2 = random_perturbation(perm)
            
            perm_copy0 = copy(perm_copy)
            perm_copy1 = copy(perm_copy)
            perm_copy2 = copy(perm_copy)
            
            perm_copy0[i, :] = perturbed_perm0
            perm_copy1[i, :] = perturbed_perm1
            perm_copy2[i, :] = perturbed_perm2
        
            # Initialize the minimum cost and the corresponding permutation
            min_cost = Inf
            best_perm = perm_copy0
            
            # Function to update the minimum cost and best permutation
            function update_best_perm(perm_copy, entry)
                sexit = Solution(instance::Instance, entry).exits
                solution = Solution(entry, sexit)
                cost = compute_cost(instance, solution, verbose=true)
                if is_feasible(instance, solution, verbose=true) && cost < min_cost
                    min_cost = cost
                    best_perm = perm_copy
                end
            end
            
            # Check each perturbed permutation
            entry0 = hcat(perm_copy0[1, :], perm_copy0[2, :], perm_copy0[3, :])
            update_best_perm(perm_copy0, entry0)
        
            entry1 = hcat(perm_copy1[1, :], perm_copy1[2, :], perm_copy1[3, :])
            update_best_perm(perm_copy1, entry1)
            
            entry2 = hcat(perm_copy2[1, :], perm_copy2[2, :], perm_copy2[3, :])
            update_best_perm(perm_copy2, entry2)
        
            # If a feasible permutation with a finite cost was found, push it to the arrays
            if min_cost < Inf
                push!(neighbor_perms, best_perm)
                push!(neighbor_costs, min_cost)
            else
                return -1, -1
            end
        end
        
        

        # Find the best neighboring solution
        min_cost_idx = argmin(neighbor_costs)
        best_neighbor = neighbor_perms[min_cost_idx]
        best_neighbor_cost = neighbor_costs[min_cost_idx]

        # If the best neighboring solution is better, update the current solution
        if best_neighbor_cost < current_cost
            counter = 0
            perms = best_neighbor
            current_cost = best_neighbor_cost

            # If the new solution is the best, update the best solution
            if best_neighbor_cost < best_cost
                best_perms = copy(best_neighbor)
                best_cost = best_neighbor_cost
            end
        else
            # If the new solution is worse, accept it with a certain probability based on temperature
            counter += 1
            if true
                acceptance_probability = exp(-(best_neighbor_cost - current_cost) / T)
                if rand() < acceptance_probability
                    perms = copy(best_neighbor)
                    current_cost = best_neighbor_cost
                end

            end
        end

        # Reduce the temperature (cooling schedule)
        T *= alpha

        # Optionally print progress
        if iter % 10 == 0
            println("Iteration: $iter, Best Cost: $best_cost, Temperature: $T")
        end
    end

    return best_perms, best_cost
end



simulated_annealing (generic function with 1 method)

In [106]:
Random.seed!(157)  # Replace 123 with your desired seed

a, b = simulated_annealing(solution, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs; T0=10000000, alpha=0.99, max_iter=1)
b

3.0

In [107]:
for i in 1:10
    perm_copy = a 
    entry = hcat(perm_copy[1, :], perm_copy[2, :], perm_copy[3, :])  # Concatenate columns to create the matrix
    sexit = (Solution(instance::Instance, entry)).exits
    solution = Solution(entry, sexit)
    a, b=simulated_annealing(solution, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs; T0=100000000, alpha=0.99, max_iter=100)
end
b

3.0

In [108]:
b

3.0

In [109]:
#here write your results we will compare them and put the best at the end
#small_1 376600.0
#small_2 357500.0
#medium_1 5.87414e6
#medium_2 5.11975e6
#large_1 23140770
#large_2 1.2143e7

In [110]:
perm_copy = a 
entry = hcat(perm_copy[1, :], perm_copy[2, :], perm_copy[3, :])  # Concatenate columns to create the matrix
sexit = (Solution(instance::Instance, entry)).exits
solution = Solution(entry, sexit)
perturbed_cost = compute_cost(instance, solution, verbose=true)
c = false
if !(is_feasible(instance, solution,verbose = true))
    c = true
end
solution.entries


5×3 Matrix{Int64}:
 1  1  1
 4  2  2
 2  3  3
 3  4  4
 5  5  5

In [111]:
current_cost = compute_cost(instance, solution, verbose=true)


3.0

In [112]:
(is_feasible(instance, solution,verbose=true))


true

In [113]:
file_path = transform_filename(filename)


"data/tiny_sol.json"

In [114]:
using JSON
# Define the path to the JSON file
###you need to change the file here too.
# Create an empty JSON object
empty_json = Dict()

# Write the empty JSON object to the file
open(file_path, "w") do file
    JSON.print(file, empty_json)
end

println("Empty JSON file created at $file_path")
write_solution(instance::Instance, solution::Solution)